In [64]:
import warnings
warnings.filterwarnings('ignore') # to avoid warnings

import random
import numpy as np
import pandas as pd
from tqdm import tqdm
import seaborn as sns
import matplotlib.pyplot as plt
import os
import sys 

"""
Sklearn Libraries
"""
from sklearn.metrics import f1_score
from sklearn.model_selection import train_test_split

"""
Transformer Libraries
"""
from transformers import BertTokenizer,  AutoModelForSequenceClassification, AdamW, get_linear_schedule_with_warmup
from transformers import BertTokenizer, BertForSequenceClassification, Trainer, TrainingArguments
from datasets import load_dataset, Dataset
"""
Pytorch Libraries
"""
import torch
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler, TensorDataset

sys.path.append('../')
from src.utilities.config_ import train_data_path
import src.utilities.utils as utils

In [78]:
# read csv
data = pd.read_csv(os.path.join(train_data_path, "finance-dataset.csv"),
                   encoding='latin-1', 
                    names=['label', 'text']).iloc[1:]

# Convert labels to integers
label_to_int = {'positive': 0, 'neutral': 1, 'negative': 2}
data['label'] = data['label'].map(label_to_int)

data

,label,text
1,1,"According to Gran , the company has no plans t..."
2,1,Technopolis plans to develop in stages an area...
3,2,The international electronic industry company ...
4,0,With the new production plant the company woul...
5,0,According to the company 's updated strategy f...
...,...,...
4842,2,LONDON MarketWatch -- Share prices ended lower...
4843,1,Rinkuskiai 's beer sales fell by 6.5 per cent ...
4844,2,Operating profit fell to EUR 35.4 mn from EUR ...
4845,2,Net sales of the Paper segment decreased to EU...


In [79]:
# Load the tokenizer
tokenizer = BertTokenizer.from_pretrained('yiyanghkust/finbert-tone')

# Load the FinBERT model
model = BertForSequenceClassification.from_pretrained('yiyanghkust/finbert-tone')

In [80]:
# Convert pandas DataFrame to Hugging Face Dataset
dataset = Dataset.from_pandas(data)

# Tokenize the dataset
def tokenize_function(examples):
    return tokenizer(examples['text'], padding='max_length', truncation=True, max_length=64)  # You can adjust the max_length as needed

tokenized_dataset = dataset.map(tokenize_function, batched=True)

# Rename the label column to "labels" for the trainer
tokenized_dataset = tokenized_dataset.rename_column("label", "labels")

# Remove unnecessary columns
tokenized_dataset = tokenized_dataset.remove_columns(["text"])

# Convert to torch tensors
tokenized_dataset.set_format("torch")

# Split the dataset into train and test sets
train_test_split = tokenized_dataset.train_test_split(test_size=0.2)
train_dataset = train_test_split['train']
test_dataset = train_test_split['test']

Map: 100%|██████████| 4846/4846 [00:01<00:00, 4042.90 examples/s]


In [81]:
# Define training arguments with adjusted logging
training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=3,
    per_device_train_batch_size=8,  # Adjust based on your memory
    per_device_eval_batch_size=16,  # Adjust based on your memory
    warmup_steps=500,
    weight_decay=0.01,
    logging_dir='./logs',
    logging_strategy="steps",  # Adjust logging strategy
    logging_steps=100,  # Log every 100 steps to reduce logging output
    log_level="error",  # Set log level to "error" to reduce output
)

In [82]:
from transformers import TrainerCallback, TrainerState, TrainerControl

class EarlyStoppingCallback(TrainerCallback):
    def __init__(self, threshold: float):
        self.threshold = threshold

    def on_step_end(self, args, state: TrainerState, control: TrainerControl, **kwargs):
        if state.log_history and 'loss' in state.log_history[-1]:
            loss = state.log_history[-1]['loss']
            if loss < self.threshold:
                control.should_training_stop = True

# Define the custom callback with your threshold
early_stopping_callback = EarlyStoppingCallback(threshold=0.1)

# Create Trainer instance
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    callbacks=[early_stopping_callback]
)

# Train the model
trainer.train()

{'loss': 4.2558, 'learning_rate': 1e-05, 'epoch': 0.21}
{'loss': 0.7412, 'learning_rate': 2e-05, 'epoch': 0.41}
{'loss': 0.6889, 'learning_rate': 3e-05, 'epoch': 0.62}
{'loss': 0.6228, 'learning_rate': 4e-05, 'epoch': 0.82}
{'loss': 0.553, 'learning_rate': 5e-05, 'epoch': 1.03}
{'loss': 0.4394, 'learning_rate': 4.4764397905759164e-05, 'epoch': 1.24}
{'loss': 0.4286, 'learning_rate': 3.9528795811518326e-05, 'epoch': 1.44}
{'loss': 0.3984, 'learning_rate': 3.429319371727749e-05, 'epoch': 1.65}
{'loss': 0.3786, 'learning_rate': 2.905759162303665e-05, 'epoch': 1.86}
{'loss': 0.2846, 'learning_rate': 2.382198952879581e-05, 'epoch': 2.06}
{'loss': 0.1559, 'learning_rate': 1.8586387434554976e-05, 'epoch': 2.27}
{'loss': 0.1146, 'learning_rate': 1.3350785340314136e-05, 'epoch': 2.47}
{'loss': 0.1215, 'learning_rate': 8.115183246073298e-06, 'epoch': 2.68}
{'loss': 0.1271, 'learning_rate': 2.879581151832461e-06, 'epoch': 2.89}
{'train_runtime': 1734.1295, 'train_samples_per_second': 6.705, 'trai

TrainOutput(global_step=1455, training_loss=0.6447465542665461, metrics={'train_runtime': 1734.1295, 'train_samples_per_second': 6.705, 'train_steps_per_second': 0.839, 'train_loss': 0.6447465542665461, 'epoch': 3.0})

In [95]:
# Predict on the test set
predictions = trainer.predict(test_dataset)
preds = np.argmax(predictions.predictions, axis=1)


In [87]:
from sklearn.metrics import accuracy_score, precision_recall_fscore_support

# Define the metric function
def compute_metrics(p):
    preds = np.argmax(p.predictions, axis=1)
    labels = p.label_ids
    precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average='weighted')
    acc = accuracy_score(labels, preds)
    return {
        'accuracy': acc,
        'f1': f1,
        'precision': precision,
        'recall': recall
    }

# Evaluate the model
metrics = compute_metrics(predictions)
print(metrics)

{'accuracy': 0.8422680412371134, 'f1': 0.8416675393188416, 'precision': 0.8412930836752625, 'recall': 0.8422680412371134}


In [58]:
test_dataset

Dataset({
    features: ['labels', 'input_ids', 'token_type_ids', 'attention_mask'],
    num_rows: 970
})

In [100]:
# Your new text
new_text = ["This is a sample financial news article. The market is looking bad.", "Market is looking great now"]

# Tokenize the new text
inputs = tokenizer(new_text, padding='max_length', truncation=True, max_length=64, return_tensors="pt")

# Move inputs to the appropriate device (e.g., CPU or GPU)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
inputs = {key: value.to(device) for key, value in inputs.items()}

# Make predictions
model.eval()
with torch.no_grad():
    outputs = model(**inputs)
    logits = outputs.logits
    predictions = torch.argmax(logits, dim=-1)

# Map predictions to labels
int_to_label = {0: 'positive', 1: 'neutral', 2: 'negative'}
predicted_labels = [int_to_label[pred.item()] for pred in predictions]

print(predicted_labels)


['negative', 'positive']


In [74]:
# Define the path to save the model and tokenizer
save_directory = "../model/finbert/"

# Save the model
model.save_pretrained(save_directory)

# Save the tokenizer
tokenizer.save_pretrained(save_directory)

('../model/finbert/tokenizer_config.json',
 '../model/finbert/special_tokens_map.json',
 '../model/finbert/vocab.txt',
 '../model/finbert/added_tokens.json')

In [94]:
# Define the path to the saved directory
save_directory = "../model/finbert/"

# Load the tokenizer
tokenizer = BertTokenizer.from_pretrained(save_directory)

# Load the model
model = BertForSequenceClassification.from_pretrained(save_directory)

In [96]:
# Define dummy training arguments (only `per_device_eval_batch_size` is relevant here)
training_args = TrainingArguments(
    output_dir='./results',
    per_device_eval_batch_size=16,  # Adjust based on your memory
)

# Create Trainer instance with the loaded model
trainer = Trainer(
    model=model,
    args=training_args
)

# Predict on the test dataset
predictions = trainer.predict(test_dataset)
preds = np.argmax(predictions.predictions, axis=1)

In [97]:
from sklearn.metrics import classification_report, accuracy_score


# Extract the true labels
true_labels = predictions.label_ids

# Generate the classification report
report = classification_report(true_labels, preds, target_names=['negative', 'neutral', 'positive'])
accuracy = accuracy_score(true_labels, preds)

print(f"Accuracy: {accuracy}")
print("Classification Report:")
print(report)

Accuracy: 0.8546391752577319
Classification Report:
              precision    recall  f1-score   support

    negative       0.80      0.77      0.78       277
     neutral       0.89      0.90      0.89       577
    positive       0.83      0.84      0.83       116

    accuracy                           0.85       970
   macro avg       0.84      0.83      0.84       970
weighted avg       0.85      0.85      0.85       970

